In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import random
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,Input,BatchNormalization,MaxPooling1D,concatenate
from keras.utils import np_utils,multi_gpu_model
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import preprocessing
from keras import backend as bek
import gc
from keras.models import load_model
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
data cell
"""
df1=pd.read_csv('/home/dp/Desktop/Bit/Bit_BP_new/datapath/pair_end_merge_SYS_random_400_split_up.csv',index_col=0)
# df1=pd.read_csv('/home/dp/Desktop/Bit/Bit_BP_new/datapath/pair_end_merge_DIA_random_400_split_down.csv',index_col=0)

In [3]:
for i,item in enumerate(df1['SYS']):
    '''
    For systolic case: Replace SYS by DIA
    '''
    if(df1['time'].iloc[i]==2):
        continue
    elif(df1['time'].iloc[i]==1):
        temp=df1[(df1['Person No']==df1['Person No'].iloc[i])&(df1['time']==2)]['SYS']
        if(temp.empty):
            continue
        else:
            print(df1['SYS'].iloc[i],temp.iloc[0])
            df1['SYS'].iloc[i]=temp.iloc[0]
    else:
        continue

124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
124.0 102.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.0 147.0
140.

In [4]:
cond2 = df1['count']!=0
df1 = df1[cond2]

#normalize
target = 'SYS'##'replace ''SYS' by 'SYS' for the case of sytolic

In [5]:
SD_last=[]# SD_last
for i,item in enumerate(df1['Person No']):
    SD_last.append(df1['SYS_last'].iloc[i]-df1['DIA_last'].iloc[i])
df1['SD_last'] = SD_last

In [6]:
'''
model cell
'''
def base_model():
    in_s = Input((406, 2))
    model_s = Convolution1D(nb_filter=256, filter_length=3) (in_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=256, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=512, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=1024, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=2048, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Flatten()(model_s)
    
    input1 = keras.layers.Input(shape=(2,))
    c = keras.layers.Concatenate(axis=-1)([model_s, input1])
    
    m = BatchNormalization(axis=-1)(c)
    m = Dense(4096,activation='relu')(m)
    m = Dense(2048,activation='relu')(m)
    out = Dense(1,activation='linear')(m)
    
    model = Model(inputs=[in_s,input1], outputs=[out])
    
    adam=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)
    model = multi_gpu_model(model,gpus=6)
    model.compile(loss='mean_squared_error', optimizer= 'adam')#'adam')
    return model

In [7]:
def test_step(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,17:423]
    X_train_s_last = train_temp.iloc[:,425:831]
    X_test_s = test_temp.iloc[:,17:423]
    X_test_s_last = test_temp.iloc[:,425:831]

    f1=['SYS']#
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshape vali data
    X_test_r_s = np.zeros((len(X_test_s), 406, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_last
    
    f2=['SYS_last','SD_last']
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape vali data
    X_test_r_f = test_temp[f2]
    
    #fit
    filepath="weights_best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True,mode='min')
    callbacks_list = [checkpoint]
    
    model= base_model()
    model.fit([X_train_r_s,X_train_r_f], Y_train, epochs=1000, batch_size=3000,callbacks=callbacks_list,verbose=0)

    
    test_pred = model.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['SYS_p']=test_pred1
    
    temp_a=0
    temp_b=0
    test_SYS_r=[]
    test_SYS_p=[]

    
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            test_SYS_r.append(test_temp['SYS'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['SYS_p'])
            test_SYS_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]

            
    print(test_SYS_p)
    print(test_SYS_r)
    del model
    bek.clear_session()
    gc.collect()
    return (test_SYS_p,test_SYS_r)

In [8]:
def test_step1(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,17:423]
    X_train_s_last = train_temp.iloc[:,425:831]
    X_test_s = test_temp.iloc[:,17:423]
    X_test_s_last = test_temp.iloc[:,425:831]

    f1=['SYS']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshape vali data
    X_test_r_s = np.zeros((len(X_test_s), 406, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_last
    
    f2=['SYS_last','SD_last']
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape vali data
    X_test_r_f = test_temp[f2]
    
    #fit
    
    model1=base_model()
    model1.load_weights('weights_best.h5', by_name = True)
    #model1.load_model('weights_best.h5')
    #print(model1.summary())

    
    test_pred = model1.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['SYS_p']=test_pred1
    
    temp_a=0
    temp_b=0
    test_SYS_r=[]
    test_SYS_p=[]

    
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            
            #administrator mode
            test_SYS_r.append(test_temp['SYS'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['SYS_p'])# t = np.median(temp['SYS_p'])
            test_SYS_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]

            
    print(test_SYS_p)
    print(test_SYS_r)
    
    del model1
    bek.clear_session()
    gc.collect()
    
    return (test_SYS_p,test_SYS_r)

In [9]:
################
#testingNo_all = [1273,543,1002,342,245,149,1888,1373,1020,1775,1598]
# testingNo_all = [1436, 1722, 2095, 2096, 343] #### Added by me
testingNo_all = [1273,543,1002,342,245,149,1888,1373,1020,1775,1598, 1436, 1722, 2095, 2096]# Added by me
################
#689
#1273,543,1002,1831,2439,1101,756,1074,1145,342,245,149,1888,1373,1020,1775,1598,1436,2095,2096,1722,2390,1483,2204,2102,2203,1390,65,2101
for testingNo in testingNo_all:

    test_p=[]
    test_r=[]
    
    test_p_best=[]
    test_r_best=[]
    #3,16
    for test_time in range(10,16):
        test  = df1[(df1['count']==testingNo)&(df1['count_time']==test_time)]
        if(test.empty):
            continue
        else:
            train = df1[(df1['count']==testingNo)&(df1['count_time']<test_time)&(df1['count_time']>(test_time-9))]
            #future_data = df1[(df1['count']==testingNo)&(df1['count_time']>=test_time)]
            #future_data_index = future_data.index
            #train = train.drop(future_data_index)
            test_index = test.index
            data_base = df1.drop(test_index)
            
            t = data_base[target]
            t = t.values.reshape(-1, 1)
            scalert = preprocessing.MinMaxScaler().fit(t)
            (temp_pred,temp_real) = test_step(train,test)
            (temp_pred_best,temp_real_best) = test_step1(train,test)
            
            test_p.append(temp_pred)
            test_r.append(temp_real)
            
            test_p_best.append(temp_pred_best)
            test_r_best.append(temp_real_best)
      
    d = {'SYS_real': test_r, 'SYS_pred': test_p}
    df_f = pd.DataFrame(data = d)
    
    ind =[]
    for j in range(2,len(df_f.index)+2):
        ind.append(j)
    
    df_f.index=ind
    
    name_temp =str(testingNo)+'-1000-256.csv'
    df_f.to_csv(name_temp)
    
    
    d_best = {'SYS_real': test_r_best, 'SYS_pred': test_p_best}
    df_f_best = pd.DataFrame(data = d_best)
    
    ind =[]
    for j in range(2,len(df_f_best.index)+2):
        ind.append(j)
    
    df_f_best.index=ind
    
    name_temp_best =str(testingNo)+'-1000-256-best.csv'
    df_f_best.to_csv(name_temp_best)
    

[28.160568, 46.360508]
[106.0, 106.0]
[7.834141, 29.513214]
[106.0, 106.0]
[107.463264, 88.38349]
[107.0, 107.0]
[104.72126, 88.070564]
[107.0, 107.0]
[71.65556, 75.16232]
[124.0, 124.0]
[71.54774, 79.95283]
[124.0, 124.0]
[127.579544, 114.13068]
[119.0, 119.0]
[130.229, 115.87484]
[119.0, 119.0]
[140.91483, 133.20828]
[125.0, 125.0]
[140.46303, 133.6991]
[125.0, 125.0]
[30.008253, 28.216784]
[136.0, 136.0]
[25.485687, 25.879612]
[136.0, 136.0]
[120.10773, 123.90911]
[109.0, 109.0]
[117.637405, 122.57843]
[109.0, 109.0]
[114.34078, 112.08658]
[119.0, 119.0]
[118.99867, 120.11777]
[119.0, 119.0]
[102.5231, 106.719536]
[122.0, 122.0]
[95.185356, 99.539246]
[122.0, 122.0]
[125.28003, 121.8608]
[115.0, 115.0]
[134.45439, 127.48834]
[115.0, 115.0]
[132.16805, 133.45856]
[134.0, 134.0]
[128.06454, 130.08604]
[134.0, 134.0]
[126.02255, 119.66461]
[117.0, 117.0]
[125.4117, 120.78075]
[117.0, 117.0]
[115.847435, 121.19108]
[116.0, 116.0]
[120.11795, 120.21336]
[116.0, 116.0]
[107.41065, 107.223

In [10]:
# from keras.models import load_model
# model = load_model('DIA/weights_best.h5')
# #model = keras.models.load_model('DIA/weights_best.hdf5')
# model.summary()